# plot_results

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt

import folders
from sites.bookcave import bookcave

## History

In [ ]:
def plot_history(path):
    # Collect history data.
    history = dict()
    with open(path, 'r') as fd:
        for line in fd:
            parts = line.split()
            key = parts[0]
            values = np.asarray(parts[1:], dtype=np.float32)
            history[key] = values
    
    # Plot loss.
    try:
        plt.plot(history['loss'], color='red', label='loss')
        plt.plot(history['val_loss'], color='blue', label='val_loss')
    except KeyError:
        print(history.keys())
    plt.legend()
    plt.title('Overall')
    plt.ylabel('Loss')
    plt.xlabel('Epochs')
    
    # Divide into categories.
    categories = [category for category in bookcave.CATEGORIES if '{}_loss'.format(category) in history]
    
    # Plot catgories.
    figure, axes = plt.subplots(nrows=len(categories), ncols=2, figsize=(1*len(categories), 18))
    for category_i, category in enumerate(categories):
        axes[category_i][0].plot(history['{}_loss'.format(category)], color='red', label='loss')
        axes[category_i][0].plot(history['val_{}_loss'.format(category)], color='blue', label='val_loss')
        axes[category_i][0].legend()
        axes[category_i][0].set_title(category)
        axes[category_i][0].set_ylabel('Loss')
        axes[category_i][0].set_xlabel('Epochs')
        try:
            axes[category_i][1].plot(history['{}_binary_accuracy'.format(category)], color='orange', label='binary_accuracy')
            axes[category_i][1].plot(history['val_{}_binary_accuracy'.format(category)], color='green', label='val_binary_accuracy')
        except KeyError:
            try:
                axes[category_i][1].plot(history['{}_categorical_accuracy'.format(category)], color='orange', label='categorical_accuracy')
                axes[category_i][1].plot(history['val_{}_categorical_accuracy'.format(category)], color='green', label='val_categorical_accuracy')
            except KeyError:
                try:
                    axes[category_i][1].plot(history['{}_accuracy'.format(category)], color='orange', label='accuracy')
                    axes[category_i][1].plot(history['val_{}_accuracy'.format(category)], color='green', label='val_accuracy')
                except KeyError:
                    axes[category_i][1].plot(history['{}_acc'.format(category)], color='orange', label='acc')
                    axes[category_i][1].plot(history['val_{}_acc'.format(category)], color='green', label='val_acc')
        axes[category_i][1].legend()
        axes[category_i][1].set_title(category)
        axes[category_i][1].set_ylabel('Accuracy')
        axes[category_i][1].set_xlabel('Epochs')
    figure.tight_layout()
    plt.show()

In [ ]:
model_names = sorted(os.listdir(folders.HISTORY_PATH))
model_names

In [ ]:
skip_names = {'.DS_Store'}
for model_name in model_names:
    if model_name in skip_names:
        continue
    fnames = sorted(os.listdir(os.path.join(folders.HISTORY_PATH, model_name)))
    for fname in fnames:
        print('{}: {}'.format(model_name, fname))
        path = os.path.join(folders.HISTORY_PATH, model_name, fname)
        plot_history(path)

## Accuracy

In [ ]:
acc_svm = [.6593, .6637, .8193, .6933, .7204, .5831, .6247, .8948]
acc_prnn = []
acc_bhan = []
acc_bnet = [.6656, .6259, .7903, .6807, .6902, .5535, .5246, .8980]
ticks = np.arange(len(bookcave.CATEGORIES))
names = [bookcave.CATEGORY_NAMES[category] for category in bookcave.CATEGORIES]
plt.bar(ticks - .2, acc_svm, width=.4)
plt.bar(ticks + .2, acc_bnet, width=.4)
plt.xticks(ticks, names, rotation=-45, ha='left')
plt.legend(['SVM', 'BookNet'])
plt.show()